In [128]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [129]:
url='https://drive.google.com/file/d/1mNbQ7KfaXDTuM17BK-ATjaj9N6aVdv2e/view?usp=sharing'
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url2)

In [130]:
df

,aspiration,doornumber,carbody,enginelocation,cylindernumber,price,engine size 1-0,horsepower 1-0,highway mpg
0,0,1,1,0,1,1,1,1,1
1,0,1,0,0,1,1,1,1,1
2,0,0,0,0,1,1,1,1,1
3,0,1,1,1,1,1,1,1,1
4,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
200,0,1,0,0,0,0,0,0,0
201,0,1,0,0,0,0,0,0,0
202,0,1,0,0,0,0,0,0,0
203,0,1,0,0,0,0,0,0,0


In [131]:
df['price']

0      1
1      1
2      1
3      1
4      1
      ..
200    0
201    0
202    0
203    0
204    0
Name: price, Length: 205, dtype: int64

In [132]:
df=df.fillna(method='ffill')

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   aspiration       205 non-null    int64
 1   doornumber       205 non-null    int64
 2   carbody          205 non-null    int64
 3   enginelocation   205 non-null    int64
 4   cylindernumber   205 non-null    int64
 5   price            205 non-null    int64
 6   engine size 1-0  205 non-null    int64
 7   horsepower 1-0   205 non-null    int64
 8   highway mpg      205 non-null    int64
dtypes: int64(9)
memory usage: 14.5 KB


In [134]:
y=df['aspiration']

In [135]:
x=df.drop('aspiration', axis=1)

In [139]:
labels = [0, 1]

In [140]:
y

0      0
1      0
2      0
3      0
4      0
      ..
200    0
201    0
202    0
203    0
204    0
Name: aspiration, Length: 205, dtype: int64

In [141]:
x

,doornumber,carbody,enginelocation,cylindernumber,price,engine size 1-0,horsepower 1-0,highway mpg
0,1,1,0,1,1,1,1,1
1,1,0,0,1,1,1,1,1
2,0,0,0,1,1,1,1,1
3,1,1,1,1,1,1,1,1
4,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...
200,1,0,0,0,0,0,0,0
201,1,0,0,0,0,0,0,0
202,1,0,0,0,0,0,0,0
203,1,0,0,0,0,0,0,0


In [142]:
x.values

array([[1, 1, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [144]:
def create_neural_net(layer_array, input_dims):
    weights = []
    biases = []
    activations = []
    
    for i in range(len(layer_array)):
        node_num = layer_array[i][0]
        weights_of_layer = []
        biases_of_layer = []
        if i == 0:
            last_layer_node_number = input_dims
        else:
            last_layer_node_number = layer_array[i-1][0]
        
        for n in range(0,node_num):
            weights_of_node = []
            for l in range(0, last_layer_node_number):
                weights_of_node.append(1) 
            weights_of_layer.append(weights_of_node)
            biases_of_layer.append(0)
            
        weights.append(weights_of_layer)
        biases.append(biases_of_layer)
        activations.append(layer_array[i][1])
    return [weights, biases, activations]

In [241]:
layer_array = [[len(labels), 'sigmoid']]
input_dims = 8
neural_net = create_neural_net(layer_array, input_dims)

print(' weights:',neural_net[0],'\n\n biases:',neural_net[1],'\n\n activations:', neural_net[2])

 weights: [[[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]] 

 biases: [[0, 0]] 

 activations: ['sigmoid']


In [146]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    
def sigmoid_deriv(x):
    return x * (1 - x)

def relu(x):
    if x < 0:
        return 0
    else:
        return x

In [147]:
def predict_ratio(data, neural_net):
    weights = neural_net[0]
    biases = neural_net[1]
    activations = neural_net[2]
    
    layer_num = len(weights)
    
    for l in range(0, layer_num):
        data = np.dot(weights[l], data)
        for t in range(len(data)):
            data[t] += biases[l][t]
        if activations[l] == 'sigmoid':
            data = sigmoid(data)
        elif activations[l] == 'relu':
            data = relu(data)
        else:
            # If not identified, do it with sigmoid
            data = sigmoid(data)
            print('activation function', activations[l], 'cannot be found. Sigmoid is used')   
    return data

def predict(data, neural_net):
    data = predict_ratio(data, neural_net)
    
    class_num = len(data)
    
    highest_class = None
    highest_class_probability = -1
    
    for i in range(0, class_num):
        if highest_class == None:
            highest_class = i
            highest_class_probability = data[i]
        elif data[i] > highest_class_probability:
            highest_class = i
            highest_class_probability = data[i]
            
    return highest_class, highest_class_probability

In [150]:
def train_network(X, Y, labels, neural_net, epochs=1000):
    for epoch in range(0, epochs):
        for d in range(0, len(X)):
            prediction = predict_ratio(X[d], neural_net)
            
            # Calculate total error per label
            true_prediction = []
            for i in range(0, len(labels)):
                true_prediction.append(0)
            true_prediction[labels.index(Y[d])] = 1
            
            errors = []
            for t in range(len(prediction)):
                errors.append(true_prediction[t] - prediction[t]) 
            adjust_deriv = errors * sigmoid_deriv(prediction)
            
            for k in range(0, len(adjust_deriv)):
                adjustment = np.dot(X[d], adjust_deriv[k])
                neural_net[0][0][k] += adjustment
    return neural_net

In [243]:
neural_net = train_network(x.values, y.values, labels, neural_net, epochs=1000)


In [244]:
for i in range(len(x.values)):
    print(predict(x.values[i], neural_net))

(0, 0.9978684849275111)
(0, 0.9821226457642208)
(0, 0.8652595485042477)
(0, 0.9992157836178959)
(0, 0.8652595485042477)
(0, 0.9821226457642208)
(0, 0.8652595485042477)
(0, 0.9978684849275111)
(0, 0.8652595485042477)
(0, 0.9992157836178959)
(0, 0.9992157836178959)
(0, 0.8652595485042477)
(0, 0.8652595485042477)
(0, 0.9821226457642208)
(0, 0.8652595485042477)
(0, 0.9826444449579748)
(0, 0.9979396911738427)
(0, 0.869179973030609)
(0, 0.869179973030609)
(0, 0.869179973030609)
(1, 0.760277285917391)
(0, 0.869179973030609)
(0, 0.7295850284140892)
(0, 0.869179973030609)
(0, 0.869179973030609)
(0, 0.9827107500451263)
(0, 0.9827107500451263)
(1, 0.760277285917391)
(0, 0.7288158575233881)
(0, 0.9826444449579748)
(1, 0.5537185272130671)
(1, 0.760277285917391)
(0, 0.9827107500451263)
(1, 0.7187952900418811)
(1, 0.760277285917391)
(1, 0.760277285917391)
(0, 0.873345832070487)
(1, 0.7187952900418811)
(0, 0.9827107500451263)
(0, 0.869179973030609)
(0, 0.9583185981804409)
(0, 0.869179973030609)
(0, 0.

In [155]:
print(len(x.values))

205


In [156]:
x.values[204]

array([1, 0, 0, 0, 0, 0, 0, 0])

In [163]:
y.values[204]

0

In [158]:
predict(x.values[204], neural_net)

(0, 0.895341471548644)

In [160]:
x.values[20]

array([0, 0, 0, 0, 0, 1, 1, 1])

In [161]:
y.values[20]

1

In [159]:
predict(x.values[20], neural_net)

(1, 0.760277285917391)

In [179]:
predict(x.values[17], neural_net)

(0, 0.869179973030609)

In [175]:
y.values[17]

1

In [183]:
predict(x.values[20], neural_net)[0]

1

In [188]:
jumlah_benar = 0
jumlah_salah = 0

for i in range(0, len(x.values)):
  if(predict(x.values[i], neural_net)[0]==y.values[i]):
    jumlah_benar+=1
  else:
    jumlah_salah+=1

print(jumlah_benar)
print(jumlah_salah)

170
35


In [192]:
accuracy = jumlah_benar/(jumlah_salah+jumlah_benar)

print("Akurasi hasil ANN : ", accuracy)

Akurasi hasil ANN :  0.8292682926829268


In [232]:
def input_parameter():
  p1 = int(input("Input '1' untuk pintu mobil lebih dari 2, dan '0' untuk 2 atau kurang : "))
  p2 = int(input("Input '1' untuk jenis mobil non-sedan, dan '0' untuk mobil sedan : "))
  p3 = int(input("Input '1' untuk lokasi engine di bagian belakang mobil, dan '0' untuk di depan mobil : "))
  p4 = int(input("Input '1' untuk jumlah cylinder mobil 5 atau lebih, dan '0' untuk kurang dari 5 : "))
  p5 = int(input("Input '1' untuk harga pasaran mobil lebih dari 30000 USD, dan '0' untuk kurang dari : "))
  p6 = int(input("Input '1' untuk engine size lebih dari 126 cc, dan '0' untuk kurang dari: "))
  p7 = int(input("Input '1' untuk horsepower lebih dari 104, dan '0' untuk kurang dari : "))
  p8 = int(input("Input '1' untuk highway mpg kurang dari 30, dan '0' untuk kurang dari : "))

  hasil_parameter = [p1, p2, p3, p4, p5, p6, p7, p8]
  return hasil_parameter

In [237]:
def prediksi():
  masukan_parameter = input_parameter()
  hasil_prediksi = predict(masukan_parameter, neural_net)

  if(hasil_prediksi[0]) == 0:
    jenis_prediksi = "Standar atau non-turbo"
  else:
    jenis_prediksi = "Turbo"
  
  print("\nPrediksi jenis mobil : ", jenis_prediksi)
  print("Perkiraan akurasi kebenaran   : ", hasil_prediksi[1]*100, "%")

In [238]:
prediksi()

Input '1' untuk pintu mobil lebih dari 2, dan '0' untuk 2 atau kurang : 0
Input '1' untuk jenis mobil non-sedan, dan '0' untuk mobil sedan : 0
Input '1' untuk lokasi engine di bagian belakang mobil, dan '0' untuk di depan mobil : 1
Input '1' untuk jumlah cylinder mobil 5 atau lebih, dan '0' untuk kurang dari 5 : 1
Input '1' untuk harga pasaran mobil lebih dari 30000 USD, dan '0' untuk kurang dari : 1
Input '1' untuk engine size lebih dari 126 cc, dan '0' untuk kurang dari: 1
Input '1' untuk horsepower lebih dari 104, dan '0' untuk kurang dari : 1
Input '1' untuk highway mpg kurang dari 30, dan '0' untuk kurang dari : 0

Prediksi jenis mobil :  Standar atau non-turbo
Perkiraan akurasi kebenaran   :  79.45668258560673 %


In [239]:
prediksi()

Input '1' untuk pintu mobil lebih dari 2, dan '0' untuk 2 atau kurang : 0
Input '1' untuk jenis mobil non-sedan, dan '0' untuk mobil sedan : 0
Input '1' untuk lokasi engine di bagian belakang mobil, dan '0' untuk di depan mobil : 0
Input '1' untuk jumlah cylinder mobil 5 atau lebih, dan '0' untuk kurang dari 5 : 0
Input '1' untuk harga pasaran mobil lebih dari 30000 USD, dan '0' untuk kurang dari : 0
Input '1' untuk engine size lebih dari 126 cc, dan '0' untuk kurang dari: 1
Input '1' untuk horsepower lebih dari 104, dan '0' untuk kurang dari : 1
Input '1' untuk highway mpg kurang dari 30, dan '0' untuk kurang dari : 1

Prediksi jenis mobil :  Turbo
Perkiraan akurasi kebenaran   :  76.02772859173909 %
